In [2]:
import torch
import sys
import os
from pathlib import Path
ROOT_FOLDER = Path(os.path.abspath("../../.."))
SRC = ROOT_FOLDER / "src"
if not SRC in sys.path:
    print("adding", SRC, "to python path for imports")
    sys.path.append(str(SRC))

import odl
import odl.contrib.torch as odl_torch
import matplotlib.pyplot as plt

from geometries.fanbeam_geometry import FlatFanBeamGeometry, DEVICE
from utils.polynomials import Legendre, Chebyshev

############## SETTINGS
beta_size=720
u_size=560
src_radius = 410.66
det_radius = 143.08
src_detector = src_radius + det_radius #-10 # 543.74 is wrong?
lims = [[-40., 40.], [-40., 40.]]
lims_reorder = [[lims[i][j] for i in range(2)] for j in range(2)]
lims_ravel = [lims[i][j] for i in range(2) for j in range(2)]
shape = [512, 512]
partition = (-56,56)
############################

phantoms = torch.stack(torch.load("../../../data/HTC2022/HTCTestPhantomsFull.pt"))[:5].to(DEVICE)
# phantoms = torch.stack(torch.load("../../../data/HTC2022/HTCTestPhantomsFull.pt"))[:5].to(DEVICE)
g = FlatFanBeamGeometry(beta_size=beta_size, u_size=u_size, src_origin=src_radius, src_detector=src_detector,
                        detector_size=partition[1]-partition[0], xy_minmax_bounds=lims_ravel, reco_shape=shape)
sinos = g.project_forward(phantoms)

projected = g.project_sinos(sinos, Chebyshev, 50, 10)

recons = g.fbp_reconstruct(sinos)
recons_projected = g.fbp_reconstruct(projected)

adding c:\Users\salom\Documents\code\KTH\KEX---CT-reconstruction\src to python path for imports


c:\Users\salom\anaconda3\envs\KEX\lib\site-packages\odl\tomo\backends\astra_cpu.py:318: RuntimeWarning: The 'astra_cpu' backend may be too slow for volumes of this size. Consider using 'astra_cuda' if your machine has an Nvidia GPU.
  warnings.warn(


TypeError: can only concatenate tuple (not "list") to tuple

In [ ]:
inspect_ind = 0
plt.subplot(121)
plt.imshow(sinos[inspect_ind].cpu())
plt.colorbar()
plt.subplot(122)
plt.imshow(projected[inspect_ind].cpu())
plt.colorbar()
plt.show()
plt.subplot(121)
plt.imshow(recons[inspect_ind].cpu())
plt.colorbar()
plt.subplot(122)
plt.imshow(recons_projected[inspect_ind].cpu())
plt.colorbar()
plt.show()